In [1]:
import ndex2.client as nc
import io
import json
import ndex2

ROOT_UUID = "bf1b0392-1adb-11ea-a741-0660b7976219"

In [2]:
client = nc.Ndex2("http://test.ndexbio.org")
summary1 = client.get_network_summary(ROOT_UUID)

In [3]:
root_cx = ndex2.create_nice_cx_from_server(server='test.ndexbio.org', uuid=ROOT_UUID)
root_cx.print_summary()

Name: NeST_final_test_20191209-152700 supporting network for NEST
Nodes: 5452
Edges: 336377
Node Attributes: 234436
Edge Attributes: 336377



In [4]:
nodes = root_cx.nodes
nodeAttr = root_cx.nodeAttributes


In [5]:
clusterMap = {}
gene_counts = {}

for key in nodeAttr:
    attr = nodeAttr[key]    
    for entry in attr:
        pointer = entry["po"]
        if pointer in clusterMap.keys():
            members = clusterMap[pointer]
        else:
            members = []

        dataType = entry["d"]
        if dataType == 'boolean':
            name = entry["n"]
            value = entry["v"]
            
            count = 0
            if name in gene_counts.keys():
                count = gene_counts[name]
            
            
            if value == 'true':
                members.append(name)
                count += 1
            
            gene_counts[name] = count

        clusterMap[pointer] = members
    
print(len(clusterMap), len(gene_counts))
print(gene_counts)

5452 43
{'Group:NEST:25': 263, 'Group:NEST:35': 86, 'Group:NEST:15': 560, 'Group:NEST:14': 435, 'Group:NEST:17': 509, 'Group:NEST:16': 153, 'Group:NEST:11': 622, 'Group:NEST:10': 794, 'Group:NEST:13': 545, 'Group:NEST:12': 378, 'Group:NEST:145': 10, 'Group:NEST:19': 229, 'Group:NEST:18': 128, 'Group:NEST:97': 18, 'Group:NEST:260': 5, 'Group:NEST:5': 685, 'Group:NEST:4': 625, 'Group:NEST:7': 571, 'Group:NEST:116': 13, 'Group:NEST:1': 828, 'Group:NEST:3': 957, 'Group:NEST:2': 1098, 'Group:NEST:9': 715, 'Group:NEST:8': 611, 'Group:NEST:28': 153, 'Group:NEST:29': 220, 'Group:NEST:66': 18, 'Group:NEST:24': 264, 'Group:NEST:43': 51, 'Group:NEST:26': 243, 'Group:NEST:20': 407, 'Group:NEST:23': 252, 'Group:NEST:33': 138, 'Group:NEST:32': 119, 'Group:NEST:277': 2, 'Group:NEST:6': 966, 'Group:NEST:325': 4, 'Group:NEST:238': 4, 'Group:NEST:239': 4, 'Group:NEST:362': 3, 'Group:NEST:217': 6, 'Group:NEST:108': 15, 'Group:NEST:237': 5}


In [6]:
# Sort
ranking = {k: v for k, v in sorted(gene_counts.items(), key=lambda item: item[1])}

In [7]:
print(ranking)

{'Group:NEST:277': 2, 'Group:NEST:362': 3, 'Group:NEST:325': 4, 'Group:NEST:238': 4, 'Group:NEST:239': 4, 'Group:NEST:260': 5, 'Group:NEST:237': 5, 'Group:NEST:217': 6, 'Group:NEST:145': 10, 'Group:NEST:116': 13, 'Group:NEST:108': 15, 'Group:NEST:97': 18, 'Group:NEST:66': 18, 'Group:NEST:43': 51, 'Group:NEST:35': 86, 'Group:NEST:32': 119, 'Group:NEST:18': 128, 'Group:NEST:33': 138, 'Group:NEST:16': 153, 'Group:NEST:28': 153, 'Group:NEST:29': 220, 'Group:NEST:19': 229, 'Group:NEST:26': 243, 'Group:NEST:23': 252, 'Group:NEST:25': 263, 'Group:NEST:24': 264, 'Group:NEST:12': 378, 'Group:NEST:20': 407, 'Group:NEST:14': 435, 'Group:NEST:17': 509, 'Group:NEST:13': 545, 'Group:NEST:15': 560, 'Group:NEST:7': 571, 'Group:NEST:8': 611, 'Group:NEST:11': 622, 'Group:NEST:4': 625, 'Group:NEST:5': 685, 'Group:NEST:9': 715, 'Group:NEST:10': 794, 'Group:NEST:1': 828, 'Group:NEST:3': 957, 'Group:NEST:6': 966, 'Group:NEST:2': 1098}


In [8]:
rank_list = list(ranking.keys())
clusterMap

def findTopGene(genes, ranking):
    top = None
    final_member = None
    for gene in genes:
        member_count = ranking[gene]
        if top is None:
            top = member_count
            final_member = gene
        elif top > member_count:
            top = member_count
            final_member = gene
    return final_member


assignedMember = {}
for key in clusterMap.keys():
    genes = clusterMap[key]
    top = findTopGene(genes, gene_counts)
    assignedMember[key] = top
    

In [9]:
for key in clusterMap.keys():
    root_cx.add_node_attribute(property_of=key, name='membership', values=clusterMap[key])
    root_cx.add_node_attribute(property_of=key, name='layoutMember', values=assignedMember[key])

In [10]:
root_cx.nodeAttributes[0]



[{'po': 0, 'n': 'Group:NEST:25', 'v': 'false', 'd': 'boolean'},
 {'po': 0, 'n': 'Group:NEST:35', 'v': 'false', 'd': 'boolean'},
 {'po': 0, 'n': 'Group:NEST:15', 'v': 'true', 'd': 'boolean'},
 {'po': 0, 'n': 'Group:NEST:14', 'v': 'false', 'd': 'boolean'},
 {'po': 0, 'n': 'Group:NEST:17', 'v': 'false', 'd': 'boolean'},
 {'po': 0, 'n': 'Group:NEST:16', 'v': 'false', 'd': 'boolean'},
 {'po': 0, 'n': 'Group:NEST:11', 'v': 'false', 'd': 'boolean'},
 {'po': 0, 'n': 'Group:NEST:10', 'v': 'false', 'd': 'boolean'},
 {'po': 0, 'n': 'Group:NEST:13', 'v': 'false', 'd': 'boolean'},
 {'po': 0, 'n': 'Group:NEST:12', 'v': 'false', 'd': 'boolean'},
 {'po': 0, 'n': 'Group:NEST:145', 'v': 'false', 'd': 'boolean'},
 {'po': 0, 'n': 'Group:NEST:19', 'v': 'true', 'd': 'boolean'},
 {'po': 0, 'n': 'Group:NEST:18', 'v': 'false', 'd': 'boolean'},
 {'po': 0, 'n': 'Group:NEST:97', 'v': 'false', 'd': 'boolean'},
 {'po': 0, 'n': 'Group:NEST:260', 'v': 'false', 'd': 'boolean'},
 {'po': 0, 'n': 'Group:NEST:5', 'v': 'fa

In [11]:
new_cx = root_cx.to_cx()

Generating CX


In [12]:
with open('newCX2.cx', 'w') as outfile:
    json.dump(new_cx, outfile)

# Community-based layout

In [28]:
# Assign cluster membership to the edges
edges = root_cx.edges
edgeAttr = root_cx.edgeAttributes

filtered = []
for key in edges:
    edge = edges[key]
    s = edge["s"]
    t = edge["t"]
    attr = edgeAttr[key]
    for entry in attr:
        filtered.append(attr)
    break
    
edge

{'@id': 29, 's': 0, 't': 3200}